In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import  seaborn as sns
%matplotlib inline

In [ ]:
data=pd.read_excel("Concrete_Dataset.xlsx")
data.head()

,Cement,Blast,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [ ]:
data.columns=['Cement',
       'Blast_Furnace_Slag',
       'Fly_Ash',
       'Water',
       'Superplasticizer',
       'Coarse_Aggregate',
       'Fine_Aggregate', 'Age',
       'Concrete_compressive_strength']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Cement                         1030 non-null   float64
 1   Blast_Furnace_Slag             1030 non-null   float64
 2   Fly_Ash                        1030 non-null   float64
 3   Water                          1030 non-null   float64
 4   Superplasticizer               1030 non-null   float64
 5   Coarse_Aggregate               1030 non-null   float64
 6   Fine_Aggregate                 1030 non-null   float64
 7   Age                            1030 non-null   int64  
 8   Concrete_compressive_strength  1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [ ]:
data.isnull().sum()

Cement                           0
Blast_Furnace_Slag               0
Fly_Ash                          0
Water                            0
Superplasticizer                 0
Coarse_Aggregate                 0
Fine_Aggregate                   0
Age                              0
Concrete_compressive_strength    0
dtype: int64

In [ ]:
data['Water_Cement_ratio']=data['Water']/data['Cement']

In [ ]:
data=data[['Superplasticizer',
       'Coarse_Aggregate', 'Fine_Aggregate', 'Age', 'Water_Cement_ratio',
       'Blast_Furnace_Slag', 'Fly_Ash',
       'Concrete_compressive_strength']]

In [ ]:
from scipy import stats 
outlier_list=[]
for c in data.columns[:-1]:
          q1=data[c].quantile(q=0.15)
          q2=data[c].quantile(q=0.85)
          L_outliers=q1-1.5*(q2-q1)
          R_outliers=q2+1.5*(q2-q1)
          outlier_list.extend(data[(data[c]<L_outliers) | (data[c]>R_outliers)].index) 
data.loc[list(set(outlier_list))]
data_outliers=data.drop(outlier_list,axis=0).reset_index(drop=True)  

In [ ]:
data.head()
X=data.drop('Concrete_compressive_strength',axis=1)
y=data['Concrete_compressive_strength']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.26,random_state=100)

In [ ]:
from sklearn.model_selection import GridSearchCV 
from xgboost import XGBRegressor
model=XGBRegressor(random_state=2,learning_rate = 0.2,
                max_depth = 2, n_estimators = 800,n_jobs=-1,reg_alpha=0.005,gamma=0.1,subsample=0.7,colsample_bytree=0.9, colsample_bylevel=0.9, colsample_bynode=0.9)
param_grid=dict()
Gridsearchmodel=GridSearchCV(model,param_grid=param_grid,cv=5,return_train_score=True)

In [ ]:
Gridsearchmodel.fit(X_train,y_train)

[05:07:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:07:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:07:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:07:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:07:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:07:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=0.9, colsample_bynode=0.9,
                                    colsample_bytree=0.9, gamma=0.1,
                                    importance_type='gain', learning_rate=0.2,
                                    max_delta_step=0, max_depth=2,
                                    min_child_weight=1, missing=None,
                                    n_estimators=800, n_jobs=-1, nthread=None,
                                    objective='reg:linear', random_state=2,
                                    reg_alpha=0.005, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=0.7, verbosity=1),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, 

In [ ]:
best_model=Gridsearchmodel.best_estimator_
best_model.fit(X_train,y_train)
y_train_pred_xg=best_model.predict(X_train)
y_test_pred_xg=best_model.predict(X_test)
print ('Train score: ',best_model.score(X_train,y_train))
print ('Test score: ',best_model.score(X_test,y_test))

[05:07:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Train score:  0.9842291894288727
Test score:  0.9306592901026743


In [ ]:
from sklearn.ensemble import  RandomForestRegressor
rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=80,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=80, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred=rf.predict(X_test)
from sklearn.metrics import r2_score
R2_score=r2_score(y_test,y_pred)
y_train_pred=rf.predict(X_train)
R2_score1=r2_score(y_train,y_train_pred)
print(R2_score1,R2_score)

0.9884392663096597 0.9014925644915265


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model2=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.3, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
param_grid=dict()
Gridsearchmodel=GridSearchCV(model,param_grid=param_grid,cv=5,return_train_score=True)

In [ ]:
model2.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.3, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
y_pred_grad_test=model2.predict(X_test)
R2_score_grad=r2_score(y_test,y_pred_grad_test)
y_pred_grad_train=model2.predict(X_train)
R2_score_grad_train=r2_score(y_train,y_pred_grad_train)
R2_score_grad_train
R2_score_grad

0.9272282457698241

In [ ]:
def rmse(predictions, targets): 
  return np.sqrt(((predictions - targets) ** 2).mean())

def mae(predictions,targets):
  return np.mean((targets-predictions)**2)

In [ ]:
# this is the rmse of Xgboot
rmse_train=rmse(y_pred_grad_train,y_train)
rmse_test=rmse(y_pred_grad_test,y_test)
# rmse_test
# rmse_train

In [ ]:
mae_train=mae(y_train_pred,y_train)
mae_test=mae(y_pred,y_test)
mae_test
# mae_train

30.942313229004494